---

OpenAI [Function Calling](https://platform.openai.com/docs/guides/function-calling) tutorial by *Dave Ebbelaar*, made into a Quarto Rendered Jupyter Notebook (by me)

- YouTube Video: <https://youtu.be/aqdWSYWC_LI?feature=shared>
- GitHub Repo: <https://github.com/daveebbelaar/langchain-experiments/tree/main?tab=readme-ov-file>

---

In [1]:
# --------------------------------------------------------------
# Import Modules
# --------------------------------------------------------------

import os
import json
import openai
from datetime import datetime, timedelta
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, ChatMessage


# --------------------------------------------------------------
# Load OpenAI API Token From the .env File
# --------------------------------------------------------------

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# Ask ChatGPT a Question


In [2]:
completion = openai.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {
            "role": "user",
            "content": "When's the next flight from Amsterdam to New York?",
        },
    ],
)

output = completion.choices[0].message.content
print(output)

I apologize, but I am an AI language model and I don't have real-time information on flight schedules. It is recommended to check with airlines or travel websites for the most up-to-date information regarding the next flight from Amsterdam to New York.


# Use OpenAI’s Function Calling Feature


In [3]:
function_descriptions = [
    {
        "name": "get_flight_info",
        "description": "Get flight information between two locations",
        "parameters": {
            "type": "object",
            "properties": {
                "loc_origin": {
                    "type": "string",
                    "description": "The departure airport, e.g. DUS",
                },
                "loc_destination": {
                    "type": "string",
                    "description": "The destination airport, e.g. HAM",
                },
            },
            "required": ["loc_origin", "loc_destination"],
        },
    }
]

user_prompt = "When's the next flight from Amsterdam to New York?"

completion = openai.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=[{"role": "user", "content": user_prompt}],
    # Add function calling
    functions=function_descriptions,
    function_call="auto",  # specify the function call
)

It automatically fills the arguments with correct info based on the prompt

Note: the function does not exist yet

In [4]:
output = completion.choices[0].message
print(output)

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "loc_origin": "AMS",\n  "loc_destination": "JFK"\n}', name='get_flight_info'), tool_calls=None)


# Add a Function


In [5]:
def get_flight_info(loc_origin, loc_destination):
    """Get flight information between two locations."""

    # Example output returned from an API or database
    flight_info = {
        "loc_origin": loc_origin,
        "loc_destination": loc_destination,
        "datetime": str(datetime.now() + timedelta(hours=2)),
        "airline": "KLM",
        "flight": "KL643",
    }

    return json.dumps(flight_info)

Use the LLM output to manually call the function

The `json.loads` function converts the string to a Python object


In [6]:
origin = json.loads(output.function_call.arguments).get("loc_origin")
destination = json.loads(output.function_call.arguments).get("loc_destination")
params = json.loads(output.function_call.arguments)
type(params)

dict

In [7]:
print(origin)
print(destination)
print(params)

AMS
JFK
{'loc_origin': 'AMS', 'loc_destination': 'JFK'}


## Call the function with arguments

In [8]:
chosen_function = eval(output.function_call.name)
flight = chosen_function(**params)

print(flight)

{"loc_origin": "AMS", "loc_destination": "JFK", "datetime": "2024-03-25 19:45:23.050928", "airline": "KLM", "flight": "KL643"}


# Add function result to the prompt for a final answer


In [9]:
# The key is to add the function output back to the messages with role: function
second_completion = openai.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "user", "content": user_prompt},
        {"role": "function", "name": output.function_call.name, "content": flight},
    ],
    functions=function_descriptions,
)
response = second_completion.choices[0].message.content
print(response)

The next flight from Amsterdam (AMS) to New York (JFK) is on March 25, 2024 at 19:45. The flight is operated by KLM and the flight number is KL643.


## Include Multiple Functions


In [10]:
# Expand on function descriptions (3 functions)

function_descriptions_multiple = [
    {
        "name": "get_flight_info",
        "description": "Get flight information between two locations",
        "parameters": {
            "type": "object",
            "properties": {
                "loc_origin": {
                    "type": "string",
                    "description": "The departure airport, e.g. DUS",
                },
                "loc_destination": {
                    "type": "string",
                    "description": "The destination airport, e.g. HAM",
                },
            },
            "required": ["loc_origin", "loc_destination"],
        },
    },
    {
        "name": "book_flight",
        "description": "Book a flight based on flight information",
        "parameters": {
            "type": "object",
            "properties": {
                "loc_origin": {
                    "type": "string",
                    "description": "The departure airport, e.g. DUS",
                },
                "loc_destination": {
                    "type": "string",
                    "description": "The destination airport, e.g. HAM",
                },
                "datetime": {
                    "type": "string",
                    "description": "The date and time of the flight, e.g. 2023-01-01 01:01",
                },
                "airline": {
                    "type": "string",
                    "description": "The service airline, e.g. Lufthansa",
                },
            },
            "required": ["loc_origin", "loc_destination", "datetime", "airline"],
        },
    },
    {
        "name": "file_complaint",
        "description": "File a complaint as a customer",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "The name of the user, e.g. John Doe",
                },
                "email": {
                    "type": "string",
                    "description": "The email address of the user, e.g. john@doe.com",
                },
                "text": {
                    "type": "string",
                    "description": "Description of issue",
                },
            },
            "required": ["name", "email", "text"],
        },
    },
]

print(function_descriptions_multiple)

[{'name': 'get_flight_info', 'description': 'Get flight information between two locations', 'parameters': {'type': 'object', 'properties': {'loc_origin': {'type': 'string', 'description': 'The departure airport, e.g. DUS'}, 'loc_destination': {'type': 'string', 'description': 'The destination airport, e.g. HAM'}}, 'required': ['loc_origin', 'loc_destination']}}, {'name': 'book_flight', 'description': 'Book a flight based on flight information', 'parameters': {'type': 'object', 'properties': {'loc_origin': {'type': 'string', 'description': 'The departure airport, e.g. DUS'}, 'loc_destination': {'type': 'string', 'description': 'The destination airport, e.g. HAM'}, 'datetime': {'type': 'string', 'description': 'The date and time of the flight, e.g. 2023-01-01 01:01'}, 'airline': {'type': 'string', 'description': 'The service airline, e.g. Lufthansa'}}, 'required': ['loc_origin', 'loc_destination', 'datetime', 'airline']}}, {'name': 'file_complaint', 'description': 'File a complaint as a 

In [11]:
def ask_and_reply(prompt):
    """Give LLM a given prompt and get an answer."""

    completion = openai.chat.completions.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": prompt}],
        # add function calling
        functions=function_descriptions_multiple,
        function_call="auto",  # specify the function call
    )

    output = completion.choices[0].message
    return output

## Scenario 1: Check flight details

In [12]:
user_prompt = "When's the next flight from Amsterdam to New York?"
print(ask_and_reply(user_prompt))

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "loc_origin": "AMS",\n  "loc_destination": "NYC"\n}', name='get_flight_info'), tool_calls=None)


### Get info for the next prompt

In [13]:
origin = json.loads(output.function_call.arguments).get("loc_origin")
destination = json.loads(output.function_call.arguments).get("loc_destination")
chosen_function = eval(output.function_call.name)
flight = chosen_function(origin, destination)

print(origin)
print(destination)
print(flight)

AMS
JFK
{"loc_origin": "AMS", "loc_destination": "JFK", "datetime": "2024-03-25 19:45:25.382829", "airline": "KLM", "flight": "KL643"}


In [14]:
flight_datetime = json.loads(flight).get("datetime")
flight_airline = json.loads(flight).get("airline")

print(flight_datetime)
print(flight_airline)

2024-03-25 19:45:25.382829
KLM


## Scenario 2: Book a new flight

In [15]:
user_prompt = f"I want to book a flight from {origin} to {destination} on {flight_datetime} with {flight_airline}"
print(ask_and_reply(user_prompt))

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "loc_origin": "AMS",\n  "loc_destination": "JFK",\n  "datetime": "2024-03-25 19:45:25.382829",\n  "airline": "KLM"\n}', name='book_flight'), tool_calls=None)


## Scenario 3: File a complaint

In [16]:
user_prompt = "This is John Doe. I want to file a complaint about my missed flight. It was an unpleasant surprise. Email me a copy of the complaint to john@doe.com."
print(ask_and_reply(user_prompt))

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n"name": "John Doe",\n"email": "john@doe.com",\n"text": "I would like to file a complaint about my missed flight. It was an unpleasant surprise. Please look into the matter and provide a suitable resolution."\n}', name='file_complaint'), tool_calls=None)


# Make It Conversational With Langchain


In [17]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0613", temperature=0)

c:\Users\rokka\anaconda3\envs\langchain-env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


## Start a conversation with multiple requests

In [18]:
user_prompt = """
This is Jane Harris. I am an unhappy customer that wants you to do several things.
First, I neeed to know when's the next flight from Amsterdam to New York.
Please proceed to book that flight for me.
Also, I want to file a complaint about my missed flight. It was an unpleasant surprise. 
Email me a copy of the complaint to jane@harris.com.
Please give me a confirmation after all of these are done.
"""

### First Response
Returns the function of the first request (`get_flight_info`)

In [19]:
first_response = llm.predict_messages(
    [HumanMessage(content=user_prompt)], functions=function_descriptions_multiple
)

print(first_response)

c:\Users\rokka\anaconda3\envs\langchain-env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `predict_messages` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


content='' additional_kwargs={'function_call': {'arguments': '{\n  "loc_origin": "AMS",\n  "loc_destination": "JFK"\n}', 'name': 'get_flight_info'}} response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 332, 'total_tokens': 358}, 'model_name': 'gpt-3.5-turbo-0613', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}


### Second Response
Returns the function of the second request (`book_flight`)

It takes all the arguments from the prompt but not the returned information


In [20]:
second_response = llm.predict_messages(
    [
        HumanMessage(content=user_prompt),
        AIMessage(content=str(first_response.additional_kwargs)),
        AIMessage(
            role="function",
            additional_kwargs={
                "name": first_response.additional_kwargs["function_call"]["name"]
            },
            content=f"Completed function {first_response.additional_kwargs['function_call']['name']}",
        ),
    ],
    functions=function_descriptions_multiple,
)

print(second_response)

content='' additional_kwargs={'function_call': {'arguments': '{\n  "loc_origin": "AMS",\n  "loc_destination": "JFK"\n}', 'name': 'get_flight_info'}} response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 387, 'total_tokens': 413}, 'model_name': 'gpt-3.5-turbo-0613', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}


### Third Response
Returns the function of the third request (`file_complaint`)

In [21]:
third_response = llm.predict_messages(
    [
        HumanMessage(content=user_prompt),
        AIMessage(content=str(first_response.additional_kwargs)),
        AIMessage(content=str(second_response.additional_kwargs)),
        AIMessage(
            role="function",
            additional_kwargs={
                "name": second_response.additional_kwargs["function_call"]["name"]
            },
            content=f"Completed function {second_response.additional_kwargs['function_call']['name']}",
        ),
    ],
    functions=function_descriptions_multiple,
)

print(third_response)

content='' additional_kwargs={'function_call': {'arguments': '{\n  "loc_origin": "AMS",\n  "loc_destination": "JFK"\n}', 'name': 'get_flight_info'}} response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 430, 'total_tokens': 456}, 'model_name': 'gpt-3.5-turbo-0613', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}


## Conversational reply at the end of requests

In [22]:
fourth_response = llm.predict_messages(
    [
        HumanMessage(content=user_prompt),
        AIMessage(content=str(first_response.additional_kwargs)),
        AIMessage(content=str(second_response.additional_kwargs)),
        AIMessage(content=str(third_response.additional_kwargs)),
        AIMessage(
            role="function",
            additional_kwargs={
                "name": third_response.additional_kwargs["function_call"]["name"]
            },
            content=f"Completed function {third_response.additional_kwargs['function_call']['name']}",
        ),
    ],
    functions=function_descriptions_multiple,
)

print(fourth_response)

content='' additional_kwargs={'function_call': {'arguments': '{\n  "loc_origin": "AMS",\n  "loc_destination": "JFK"\n}', 'name': 'get_flight_info'}} response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 473, 'total_tokens': 499}, 'model_name': 'gpt-3.5-turbo-0613', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}
